<a href="https://colab.research.google.com/github/FilipKatulski/natural-language-processing/blob/dev-task6/6-classification/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install datasets
!pip install transformers
!pip install fasttext
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 30.6 MB/s 
     |████████████████████████████████| 132 kB 75.4 MB/s 
     |████████████████████████████████| 182 kB 74.6 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 127 kB 64.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 19.8 MB/s 
     |████████████████████████████████| 7.6 MB 68.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 212 kB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fastt

In [ ]:
from datasets import load_dataset, load_metric
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score
from sklearn.naive_bayes import GaussianNB
from transformers import pipeline, AutoTokenizer, Trainer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments
import fasttext
import numpy as np
import pandas as pd
from lime.lime_text import LimeTextExplainer


## Get acquainted with the data of the Polish Cyberbullying detection dataset. Pay special attention to the distribution of the positive and negative examples in the first task as well as distribution of the classes in the second task.

In [ ]:
dataset_task1 = load_dataset("poleval2019_cyberbullying", "task01")
dataset_task2 = load_dataset("poleval2019_cyberbullying", "task02")


Generating train split:   0%|          | 0/10041 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset poleval2019_cyberbullying downloaded and prepared to /root/.cache/huggingface/datasets/poleval2019_cyberbullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10041 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset poleval2019_cyberbullying downloaded and prepared to /root/.cache/huggingface/datasets/poleval2019_cyberbullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Split data to train and test subsets

dataset_task1_train = dataset_task1['train']
dataset_task1_test = dataset_task1['test']
dataset_task2_train = dataset_task2['train']
dataset_task2_test = dataset_task2['test']

In [ ]:
dataset_task1_train[:10]

{'text': ['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
  '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
  '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
  '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
  'Odrzut natychmiastowy, kwaśna mina, mam problem',
  'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
  '@anonymized_account No nie ma u nas szczęścia 😉',
  '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
  '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
  '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account'],
 'label': [

In [ ]:
dataset_task2_train[:10]

{'text': ['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
  '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
  '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
  '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
  'Odrzut natychmiastowy, kwaśna mina, mam problem',
  'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
  '@anonymized_account No nie ma u nas szczęścia 😉',
  '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
  '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
  '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account'],
 'label': [

## Train the following classifiers on the training sets (for the task 1 and the task 2)

### Bayesian classifier with TF * IDF weighting.


In [ ]:
columns = ["classifier", "accuracy", "precision", "recall", "f1"]

def to_scores_df(model, scores):
    return pd.DataFrame(data=[[
        model,
        scores[columns[1]],
        scores[columns[2]],
        scores[columns[3]],
        scores[columns[4]],
    ]], columns=columns)

scores_task1 = pd.DataFrame(data=[], columns=columns)
scores_task2 = pd.DataFrame(data=[], columns=columns)



In [ ]:
def tf_idf(train, test):    
    vectorizer = TfidfVectorizer()
    vectorizer.fit(train)
    return vectorizer.transform(train).toarray(), vectorizer.transform(test).toarray()

dataset_task1_train_tfidf, dataset_task1_test_tfidf = tf_idf(dataset_task1_train['text'], dataset_task1_test['text'])
dataset_task2_train_tfidf, dataset_task2_test_tfidf = tf_idf(dataset_task2_train['text'], dataset_task2_test['text'])

From sklearn documentation: "GaussianNB implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian."

In [ ]:
gnb1 = GaussianNB()
gnb1.fit(dataset_task1_train_tfidf, dataset_task1_train['label'])
gnb2 = GaussianNB()
gnb2.fit(dataset_task2_train_tfidf, dataset_task2_train['label'])


GaussianNB()

In [ ]:
def evaluate_task1(predicted, actual):
    return {"accuracy": accuracy_score(predicted, actual), "precision": precision_score(predicted, actual), "recall": recall_score(predicted, actual), "f1": f1_score(predicted, actual)} 

# Using macro F1 average.
def evaluate_task2(predicted, actual):
    return {"accuracy": accuracy_score(predicted, actual), "precision": precision_score(predicted, actual, average='macro'), "recall": recall_score(predicted, actual, average='macro'), "f1": f1_score(predicted, actual, average='macro')} 
 

In [ ]:
gnb1_scores = evaluate_task1(gnb1.predict(dataset_task1_test_tfidf), dataset_task1_test['label'])
gnb2_scores = evaluate_task2(gnb2.predict(dataset_task2_test_tfidf), dataset_task2_test['label'])

scores_task1 = scores_task1.append(to_scores_df("GaussianNaiveBayes", gnb1_scores))
scores_task2 = scores_task2.append(to_scores_df("GaussianNaiveBayes", gnb2_scores))


In [ ]:
gnb1_scores
     

{'accuracy': 0.782,
 'precision': 0.29850746268656714,
 'recall': 0.24390243902439024,
 'f1': 0.2684563758389261}

In [ ]:
gnb2_scores


{'accuracy': 0.787,
 'precision': 0.4081828647301029,
 'recall': 0.40132515731936985,
 'f1': 0.3968305029876156}

In [ ]:
scores_task1

,classifier,accuracy,precision,recall,f1
0,GaussianNaiveBayes,0.782,0.298507,0.243902,0.268456


In [ ]:
scores_task2

,classifier,accuracy,precision,recall,f1
0,GaussianNaiveBayes,0.787,0.408183,0.401325,0.396831


### Fasttext text classifier


Fasttext.train_supervised requires '.txt' files as input.

In [ ]:
def to_fasttext_input(dataset, filename):
    with open(filename, "w") as f:
        for label, text in zip(dataset['label'], dataset['text']):
            f.write(f"__label__{label} {text}\n")

to_fasttext_input(dataset_task1_train, 'fasttext_train1.txt')
to_fasttext_input(dataset_task1_test, 'fasttext_test1.txt')
to_fasttext_input(dataset_task2_train, 'fasttext_train2.txt')
to_fasttext_input(dataset_task2_test, 'fasttext_test2.txt')


In [ ]:
fasttext_model1 = fasttext.train_supervised('fasttext_train1.txt')
fasttext_model2 = fasttext.train_supervised('fasttext_train2.txt')


TIP: Do not report Precision, Recall and F1 score

In [ ]:
def fasttext_scores(result):
    # Not reporting prec, rec, f1 as suggested in the tips in the lab's description
    return { "accuracy": result[1], "precision": None, "recall": None, "f1": None } 

fasttext_model1_scores = fasttext_scores(fasttext_model1.test('fasttext_test1.txt'))
fasttext_model2_scores = fasttext_scores(fasttext_model2.test('fasttext_test2.txt'))
scores_task1 = scores_task1.append(to_scores_df("fastText", fasttext_model1_scores))
scores_task2 = scores_task2.append(to_scores_df("fastText", fasttext_model2_scores))
  

In [ ]:
fasttext_model1_scores

{'accuracy': 0.873, 'precision': None, 'recall': None, 'f1': None}

In [ ]:
fasttext_model2_scores

{'accuracy': 0.868, 'precision': None, 'recall': None, 'f1': None}

In [ ]:
scores_task1

,classifier,accuracy,precision,recall,f1
0,GaussianNaiveBayes,0.782,0.298507,0.243902,0.268456
0,fastText,0.873,None,None,None


In [ ]:
scores_task2

,classifier,accuracy,precision,recall,f1
0,GaussianNaiveBayes,0.787,0.408183,0.401325,0.396831
0,fastText,0.868,None,None,None


### Transformer classifier (take into account that a number of experiments should be performed for this model).

In [ ]:
def fine_tuned(model_name, dataset, expected_labels, num_epochs=2):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenized_dt = dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=expected_labels)
    
    training_args = TrainingArguments(
        output_dir='./results',
        learning_rate=0.00002,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dt["train"],
        eval_dataset=tokenized_dt["test"],
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()
    return model
     

ImportError: You need to install sacremoses to use HerbertTokenizer. See https://pypi.org/project/sacremoses/ for installation.


In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 32.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=fb5795c6e4661a5337854843aec9ca958abc086188ee23f3b98702574f347d28
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
herbert_fine_tuned1 = fine_tuned("allegro/herbert-base-cased", dataset_task1, expected_labels=2)
herbert_fine_tuned1.save_pretrained("herbert-base-cased-bullying")


Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/907k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

Epoch,Training Loss,Validation Loss
1,0.255900,0.350258
2,0.189200,0.272788


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.jso

In [ ]:
herbert_fine_tuned2 = fine_tuned("allegro/herbert-base-cased", dataset_task2, expected_labels=3, num_epochs=3)
herbert_fine_tuned2.save_pretrained("herbert-base-cased-bullying_2")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 

Epoch,Training Loss,Validation Loss
1,0.298000,0.418627
2,0.216100,0.495861
3,0.166200,0.450020


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.jso

In [ ]:
def compute_metrics_singleclass(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return evaluate_task1(pred, labels)

def compute_metrics_multiclass(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return evaluate_task2(pred, labels)  # macro F1 

def evaluate_transformers(model, dataset, tokenizer_name, compute_metrics):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenized_dt = dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True)
    trainer = Trainer(model=model,
                      eval_dataset=tokenized_dt["test"],
                      tokenizer=tokenizer,
                      compute_metrics=compute_metrics)
    return trainer.evaluate()
  

In [ ]:
def transformers_to_scores(eval_res):
    return {"accuracy": eval_res["eval_accuracy"], "precision": eval_res["eval_precision"], "recall": eval_res["eval_recall"], "f1": eval_res["eval_f1"]} 

herbert_fine_tuned1_score = transformers_to_scores(
    evaluate_transformers(herbert_fine_tuned1, dataset_task1, "allegro/herbert-base-cased", compute_metrics_singleclass))
herbert_fine_tuned2_score = transformers_to_scores(
    evaluate_transformers(herbert_fine_tuned2, dataset_task2, "allegro/herbert-base-cased", compute_metrics_multiclass))

scores_task1 = scores_task1.append(to_scores_df("transformers_herbert-cased", herbert_fine_tuned1_score))
scores_task2 = scores_task2.append(to_scores_df("transformers_herbert-cased", herbert_fine_tuned2_score))



loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
You're using a HerbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
You're using a HerbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Compare the results of classification on the test set. Select the appropriate measures (from accuracy, F1, macro/micro F1, MCC) to compare the results.



Selected macroF1 for Task2, as it has 3 different classes and imbalanced dataset.

In [ ]:
scores_task1


,classifier,accuracy,precision,recall,f1
0,GaussianNaiveBayes,0.782,0.298507,0.243902,0.268456
0,fastText,0.873,None,None,None
0,transformers_herbert-cased,0.914,0.447761,0.833333,0.582524


In [ ]:
scores_task2


,classifier,accuracy,precision,recall,f1
0,GaussianNaiveBayes,0.787,0.408183,0.401325,0.396831
0,fastText,0.868,None,None,None
0,transformers_herbert-cased,0.897,0.481181,0.573795,0.510034


## Select 1 TP, 1 TN, 1 FP and 1 FN from your predictions (for the best classifier) and compare the decisions of each classifier on these examples using LIME.

In [ ]:
def predictions(model, dataset):
    lime_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
    tokenized = [lime_tokenizer(x, truncation=True) for x in dataset]
    trainer = Trainer(model=model, tokenizer=lime_tokenizer)
    return trainer.predict(tokenized)


In [ ]:
pred = predictions(herbert_fine_tuned1, dataset_task1['test']['text'])[0]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

In [ ]:
pred_labels = np.argmax(pred, axis=1)
real_labels = dataset_task1["test"]["label"]

zipped = list(zip(pred_labels, real_labels))
tp = zipped.index((1, 1)) 
tn = zipped.index((0, 0))
fp = zipped.index((1, 0))
fn = zipped.index((0, 1))



In [ ]:
dataset_task1["test"][tp]

{'text': '@anonymized_account Dokładnie, pisdzielstwo nie ma prawa rozpierdalać systemu,  sądownictwa nie mając większości',
 'label': 1}

In [ ]:
dataset_task1["test"][tn]

{'text': '@anonymized_account Spoko, jak im Duda z Morawieckim zamówią po pięć piw to wszystko będzie ok.',
 'label': 0}

In [ ]:
dataset_task1["test"][fp]

{'text': '@anonymized_account Jestem ukrainskim żydem z polskim obywatelstwem tnij może jedna ci starczy',
 'label': 0}

In [ ]:
dataset_task1["test"][fn]

{'text': '@anonymized_account Tej szmaty się nie komentuje', 'label': 1}

In [ ]:
def explain(text):
    exp = LimeTextExplainer(class_names=["neutral", "bullying"]).explain_instance(text, lambda x: predictions(herbert_fine_tuned1, x)[0],\
                                                                                  num_features=10)
    return exp.as_list()

tp_explained = explain(dataset_task1["test"][tp]["text"])
tn_explained = explain(dataset_task1["test"][tn]["text"])
fp_explained = explain(dataset_task1["test"][fp]["text"])
fn_explained = explain(dataset_task1["test"][fn]["text"])
     

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3df0dc2297/config.json
Model config BertConfig {
  "_name_or_path": "allegro/herbert-base-cased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "HerbertTokenizerFast",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--allegro--herbert-base-cased/snapshots/50e33e0567be0c0b313832314c586e3d

The more positive value of the word the more it pushes the sentence towards the "bullying" class.

In [ ]:
print("Explain true positive", tp_explained)
print("Explain true negative", tn_explained)
print("Explain false positive", fp_explained)
print("Explain false negative", fn_explained)


Explain true positive [('pisdzielstwo', 2.659216580538933), ('anonymized_account', 0.30410310506805466), ('rozpierdalać', 0.2231332883966804), ('nie', -0.21547734815467143), ('Dokładnie', -0.15245681685040055), ('prawa', -0.08826783534140129), ('sądownictwa', 0.07586703460381045), ('ma', -0.06485400136937262), ('mając', 0.05529496452131282), ('większości', -0.05301258082320065)]
Explain true negative [('Morawieckim', 0.3345445631995327), ('piw', -0.2038510781184819), ('ok', -0.16311310096692294), ('Spoko', -0.15726466518921994), ('Duda', 0.12024223438324738), ('pięć', -0.09150321026248309), ('to', -0.06534730504897722), ('im', 0.06119036808569243), ('anonymized_account', 0.054656580211959786), ('jak', 0.05226631663318514)]
Explain false positive [('żydem', 2.2426894354180114), ('ci', 0.5709097054926481), ('ukrainskim', 0.36258721781877135), ('tnij', 0.3311086929514432), ('starczy', 0.3015115198686023), ('anonymized_account', 0.2550533818028052), ('może', -0.2206878626385482), ('Jestem'

## Answer the following questions:
 - Which of the classifiers works the best for the task 1 and the task 2.

  Transformers with Herbert LM give the best results for both Tasks.

 - Did you achieve results comparable with the results of PolEval Task?

  Task 1:

  System name,	Precision, Recall, F1,	Accuracy
  
  n-waves ULMFiT,	66.67,	52.24,	58.58,	90.10

  Tranformers method got better results for Task 1

  Task 2:

  System name,	Micro-Average F1, Macro-Average F1
  
  model1-svm,	87.60,	51.75

  Got worse results that.

 - Did you achieve results comparable with the Klej leaderboard?

  https://klejbenchmark.com/leaderboard/ does not work as of Thursday 8 Dec 2022 10pm. 

 - Describe strengths and weaknesses of each of the compared algorithms.

  Fasttext method was the fastest method, but Transformers gave the best results. Fasttext does not return F1, precision and recall (at least it is suggested that we should not use these values)

 - Do you think comparison of raw performance values on a single task is enough to assess the value of a given algorithm/model?

 Yes but these datasets are actually quite small and could not be useful with the exterior, additional data.

 - Did LIME show that the models use valuable features/words when performing their decision?

 The higher the value of a word the more "bullying" is the sentence. 


